In [68]:
import math
outputs = [0, 0, 2, 2, 1] # X_1 .. X_N
transition_prob = [[0.55, 0.25, 0.2],\
                   [0.15, 0.7, 0.15],\
                   [0.2, 0.3, 0.4]]
emission_prob = [[0.1, 0.3, 0.6],\
                 [0.2, 0.5, 0.3],\
                 [0.6, 0.3, 0.1]]
start_prob = [0.3, 0.3, 0.4]

In [57]:
import math

class HMM:
  def __init__(self, transition_prob: list[list[int]], emission_prob: list[list[int]], start: list[int]):
    # P(Y_(t+1) | Y_t)
    self.transition_prob = transition_prob

    # P(X_t | Y_t)
    self.emission_prob = emission_prob

    self.start = start

  def joint_prob(self, outputs: list[int], hiddens: list[int]) -> float:
    res = start_prob[hiddens[0]]
    for i in range(len(outputs)):
      # Multiply in transition prob
      if i != 0:
        res *= transition_prob[hiddens[i-1]][hiddens[i]]
      res *= emission_prob[hiddens[i]][outputs[i]]
    return res


  def conditional_weights(self, outputs: list[int], hiddens: list[int], t: int) -> list[float]:
    # Returns P(Yi = y | Yi-1 = yi-1, Yi+1 = yi+1, Xi = xi)
    res = []
    for y in range(3): # iterates through y = 0, 1, 2
      # Adding one instance to result
      inst = 1
      # First term: P(Yi = y | Yi-1 = yi-1) if t!=0 else P(Y0=y)
      first_term = transition_prob[hiddens[t-1]][y] if t != 0 else start_prob[y]
      inst *= first_term
      # Second term: P(Yi+1 = yi+1 | Yi = y) -- omit if i = n-1
      if t != len(hiddens) - 1:
        inst *= transition_prob[y][hiddens[t+1]]
      # Third term: P(Xi = xi | Y = y)
      inst *= emission_prob[y][outputs[t]]
      res.append(inst)
    return res

In [69]:
hmm = HMM(transition_prob, emission_prob, start_prob)

In [70]:
# Test cases
print(hmm.joint_prob([0,0,2,2,1], [1,1,1,1,1]))
# about 0.000129654
print(hmm.joint_prob([0,0,2,2,1], [2,2,0,0,1]))
# 0.00028512

0.00012965399999999993
0.00028512000000000003


In [71]:
import itertools

def get_most_likely_hidden_states(hmm: HMM, outputs: list[int]):
  # TODO
  res_prob = [] # (hidden states, conditional prob)
  all_hiddens = []
  # iterate over possible hidden states
  for hiddens in itertools.product([0,1,2],repeat=5):
    hiddens = list(hiddens) # convert to list
    all_hiddens.append(hiddens)

  # Calculate P(x1,..,xn) by summing out Ys
  denom = sum(hmm.joint_prob(outputs, h) for h in all_hiddens)

  for h in all_hiddens:
    prob = hmm.joint_prob(outputs, h) / denom
    res_prob.append((prob, h))

  sorted_res = sorted(res_prob, key=lambda x: x[0], reverse=True)

  for i in range(10):
    print("Rank " + str(i+1) + ": Y states " + str(sorted_res[i][1]) + " with conditional probability of " + str(sorted_res[i][0]))


In [72]:
get_most_likely_hidden_states(hmm, outputs)

Rank 1: Y states [2, 2, 1, 1, 1] with conditional probability of 0.10142053368734852
Rank 2: Y states [2, 2, 0, 0, 0] with conditional probability of 0.10017864960138102
Rank 3: Y states [2, 2, 0, 0, 1] with conditional probability of 0.0758929163646826
Rank 4: Y states [2, 1, 1, 1, 1] with conditional probability of 0.05916197798428662
Rank 5: Y states [2, 2, 0, 1, 1] with conditional probability of 0.04829549223207073
Rank 6: Y states [2, 2, 0, 0, 2] with conditional probability of 0.03642859985504764
Rank 7: Y states [1, 1, 1, 1, 1] with conditional probability of 0.034511153824167196
Rank 8: Y states [2, 0, 0, 0, 0] with conditional probability of 0.022957607200316484
Rank 9: Y states [2, 2, 1, 0, 0] with conditional probability of 0.020491087418464296
Rank 10: Y states [2, 2, 2, 1, 1] with conditional probability of 0.01931819689282829


# Gibbs Sampling

In [73]:
# Test cases
print(hmm.conditional_weights([0,0,2,2,1], [1,1,1,1,1], 2))
# about [0.0225, 0.147, 0.0045]
print(hmm.conditional_weights([0,0,2,2,1], [2,2,0,0,1], 2))
# about [0.066, 0.0135, 0.00800000000000000]

[0.0225, 0.14699999999999996, 0.0045]
[0.066, 0.0135, 0.008000000000000002]


In [87]:
# Perform a single Gibbs sample Y_1 .. Y_N ~ P(Y_1 .. Y_N | X_1 .. X_N)
import random

def gibbs_sample(hmm: HMM, outputs: list[int]) -> list[list[int]]:
  # Run outer loop for 1000 iterations, then 5000 take a sample of y0...yn-1 and add to res
  # Use random.choices(population=population, weights=weights, k=1)[0] to sample a single value
  N = len(outputs)
  ys = [0] * N
  res = [None] * 5000

  for t in range(6000):
    # Making the step
    for i in range(N):
      weights = hmm.conditional_weights(outputs, ys, i)
      ys[i] = random.choices(population=[i for i in range(len(weights))], weights=weights, k=1)[0]
    if t >= 1000:
      res[t-1000] = [y for y in ys]

  return res

In [92]:
import collections
def estimate_likely_hidden_states(hmm: HMM, outputs: list[int]):
  res_prob = [] # (hidden states, conditional prob)
  all_hiddens = {}
  # iterate over possible hidden states
  for hiddens in itertools.product([0,1,2],repeat=5):
    hiddens = list(hiddens) # convert to list
    all_hiddens[tuple(hiddens)] = 0
  samples = gibbs_sample(hmm, outputs)
  for sample in samples:
    all_hiddens[tuple(sample)] += 1
  sorted_samples = sorted(all_hiddens, key=lambda x: all_hiddens[x], reverse=True)
  for i in range(10):
    print(sorted_samples[i])

In [93]:
estimate_likely_hidden_states(hmm, outputs)

(2, 2, 1, 1, 1)
(2, 2, 0, 0, 0)
(2, 2, 0, 0, 1)
(2, 1, 1, 1, 1)
(2, 2, 0, 1, 1)
(1, 1, 1, 1, 1)
(2, 2, 0, 0, 2)
(2, 0, 0, 0, 0)
(2, 1, 0, 0, 0)
(2, 2, 2, 1, 1)
